<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/MonteCarlo/MC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monte Carlo Simulations with LiBELa #

This Colab uses pyLiBELa for Monte Carlo simulations. In this simulations, the ligand is samples in a Metropolis MC scheme bound in a protein active site.

The required files are:
* a protein structure in MOL2 format;
* a ligand structure in MOL2 format;


In [1]:
#@title Downloading dependencies {display-mode: "form"}

%%capture
! apt-get install python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev
!pip install ipython-autotime
%load_ext autotime
!pip3 install condacolab
import condacolab
import timeit
from google.colab import files
condacolab.install()
!mamba install openbabel

time: 4.23 s (started: 2023-10-10 12:33:24 +00:00)


In [2]:
#@title Installing pyLiBELa {display-mode: "form"}

%%capture
! git clone https://github.com/alessandronascimento/pyLiBELa.git
! mv pyLiBELa/src src
! mv pyLiBELa/test test
! rm -rf pyLiBELa
! mkdir -p obj
! rm -f Makefile*
! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
! make -j2

time: 2min 37s (started: 2023-10-10 12:33:28 +00:00)


In [3]:
#@title Importing pyLiBELa {display-mode: "form"}
try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')

pyLiBELa is imported!
time: 39.4 ms (started: 2023-10-10 12:36:05 +00:00)


In [4]:
#@title Upload Files {display-mode: "form"}
rec_upload = files.upload()
for fn in rec_upload.keys():
  print("File {name} uploaded!".format(name=fn))

Saving MR.mol2.gz to MR.mol2.gz
Saving AS4.mol2.gz to AS4.mol2.gz
File MR.mol2.gz uploaded!
File AS4.mol2.gz uploaded!
time: 9.87 s (started: 2023-10-10 12:37:47 +00:00)


In [5]:
Input = PARSER()

# Some default options:
# 1. We are using two processors for Grid calculations;
#

Input.generate_conformers = True;
Input.dock_parallel = True;
Input.parallel_jobs = 2;
Input.eq_mode = True;
Input.write_grids = False
Input.load_grid_from_file = False;
Input.grid_prefix = 'McGrid'
Input.use_grids = True
Input.write_mol2 = False
Input.atom_limit = 100
    #atom limit not counting H

# Energy Calculation Parameters:
scoring_function = "0" #@param ["0", "1", "2", "3"]
Input.dielectric_model = "r" #@param ["r", "constant"]
Input.scoring_function = int(scoring_function)
grid_dimension = 30.0 #@param {type:"number"}
Input.grid_spacing = 0.5 #@param {type:"number"}
Input.solvation_alpha = 0.1 #@param {type:"number"}
Input.solvation_beta = -0.005 #@param {type:"number"}

# Optimization parameter:
Input.min_tol = 1E-5;
Input.min_delta = 1E-3;
Input.dock_min_tol = 1E-5;
search_box = 20.0 #@param {type:"number"}
Input.timeout = 20
Input.min_timeout = 20
Input.overlay_optimizer = "mma"
Input.energy_optimizer = "mma"
if (Input.scoring_function < 3):
  delta = 2.5 #@param {type:"number"}
  Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
  delta_es = 2.5 #@param {type:"number"}
  Input.deltaij_es6 = pow(delta_es, 6);
  Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.number_steps=5000 #@param {type:"number"}
Input.temp=300.0 #@param {type:"number"}
Input.rotation_step=1.25 #@param {type:"number"}
Input.torsion_step=1.25 #@param {type:"number"}
Input.sample_torsions=True;
Input.eq_steps=1000000 #@param {type:"number"}
Input.ligsim=True;
Input.mc_stride=1 #@param {type:"number"}
Input.seed=1423 #@param {type:"number"}
Input.output="MC" # @param {type:"string"}
Input.generate_conformers=False;

Input.lig_mol2='AS4.mol2.gz' # @param {type:"string"}
Input.reflig_mol2='AS4.mol2.gz' # @param {type:"string"}
Input.rec_mol2='MR.mol2.gz' # @param {type:"string"}


Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;
Input.x_dim, Input.y_dim, Input.z_dim = grid_dimension, grid_dimension, grid_dimension;


time: 3.08 ms (started: 2023-10-10 12:39:13 +00:00)


In [ ]:
Rec = Mol2(Input, Input.rec_mol2)
Lig = Mol2(Input, Input.lig_mol2)
RefLig = Mol2(Input, Input.reflig_mol2)
Writer = WRITER(Input)
Coord = COORD_MC()
center = Coord.compute_com(Lig)
Grids = Grid(Input, Writer, Rec, center)
EqMC = MC(Lig, Input, Writer)
print('Grids computed. Starting MC calculation...')
EqMC.run(Grids, RefLig, Lig, Lig.xyz, Input, Input.temp)
if (Input.ligsim):
  EqMC.ligand_run(Lig, Lig, Lig.xyz, Input, Input.temp)


print()
print()
print("Final Binding Energies:")
print("     DeltaU = <Ebound> - <Efree> = %7.3f kcal/mol" % EqMC.average_deltaE)
print("     TdS = - (<TSbound> - <TSfree>) = %7.3f kcal/mol" % (EqMC.boundTS-EqMC.freeTS));
print("     Binding Energy = <dU> - TdS = %7.3f kcal/mol" % (EqMC.average_deltaE - (EqMC.boundTS-EqMC.freeTS)));
print()
print()